In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# %% load packages
import locale
import sys
import os
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.getcwd()

'c:\\Users\\amaguaya\\OneDrive - Kienzle Automotive GmbH\\Desktop\\DLiE\\repo\\DLiE_forecast_13_06_25\\srs\\notebooks'

In [5]:
import sys
sys.path.append('C:/Users/amaguaya/OneDrive - Kienzle Automotive GmbH/Desktop/DLiE/repo/DLiE_forecast_13_06_25')

import os
print("Working directory:", os.getcwd())



Working directory: c:\Users\amaguaya\OneDrive - Kienzle Automotive GmbH\Desktop\DLiE\repo\DLiE_forecast_13_06_25\srs\notebooks


In [6]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, SimpleMLP, train_mlp, \
  prepare_train_test_tensors, build_mlp_rolling_forecasts, tune_mlp_hyperparameters, \
  DST_trafo, prepare_dataset_tensor_modified

from srs.utils.our_utils import run_forecast_step, run_forecast_step_modified
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset



### models_24h fitting for all no1-no5 regions separately.

In [79]:
'''
  eval/test periods
  2023 - 365 days
  2024 - 366 days
'''
# rmse  # -- 31.68, rmse: 8.79, 8.798 with rolling, 8.77 use n+180, -90... 8.7978; -120 (4months):9.51 
# -30 (1 month)8.72, -60 (2mothns), 8.703, if remove 2019. 8.73

# id_init_eval-1
# data_t.shape, train_t.shape, train_dates.shape
# 2192-751

# len(price_t[24:-24])
# len(data_t[:,:,0])

'\n  eval/test periods\n  2023 - 365 days\n  2024 - 366 days\n'

In [80]:
# len(price_t[:-1])
# len(full_dates_), len(data_t[:,:,0].cpu().numpy())

In [ ]:
# # Step 1: Flatten the time series (days × hours → 1D)
# flattened_series = data_t[:, :, 0].reshape(-1).cpu().numpy()

# # Step 2: Create a matching hourly datetime index
# # There are 24 hours per day, so total points = 2192 days × 24 = 52608
# start_datetime = pd.to_datetime(INIT_DATE_EXPERIMENTS)
# end_datetime = pd.to_datetime(FINAL_DATE_EXPERIMENTS) + pd.Timedelta(hours=23)
# datetime_index = pd.date_range(start=start_datetime, end=end_datetime, freq='H')

# # Check: both must match in length
# assert len(flattened_series) == len(datetime_index)

# # Step 3: Create the Series and plot
# price_series_flat = pd.Series(flattened_series, index=datetime_index)

# plt.figure(figsize=(15, 4))
# price_series_flat[-(24*7*0)-(2*24):].plot()
# plt.title("Electricity Price (Hourly)")
# plt.xlabel("Time")
# plt.ylabel("Price")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# [1,2,5,88,22,4,9,8,2,5,7,10,4,5,7,8,44,0,99,77][-(3*2):-3]

# Create mask: keep only valid forecast rows (no NaNs across the whole 24h horizon)
# valid_mask = ~torch.isnan(forecasts_zone[:, :, 0]).any(dim=1)

# # Apply mask to both forecast and true values
# valid_forecast = forecasts_zone[valid_mask, :, 0]
# valid_true     = true_vals[valid_mask, :]

# # Compute RMSE only on valid rows
# diff = valid_forecast - valid_true
# rmse = torch.sqrt((diff ** 2).mean()).item()
# print(f"Filtered RMSE: {rmse:.4f}")


Filtered RMSE: 14.3937


In [ ]:
# set the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **************************************
# define dates for training and evaluation 
# **************************************
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE = '2023-01-01'
FINAL_DATE_EXPERIMENTS = '2024-12-31'
n_days_test = (pd.to_datetime(FINAL_DATE_EXPERIMENTS) - pd.to_datetime(INIT_TEST_DATE)).days + (1) # additional adjustment

repo_root = Path.cwd().parents[1]
mapcodes = ["NO1"] # ["NO1","NO2","NO3","NO4","NO5"]
maps_dict = {}

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df = pd.read_csv(csv_path, parse_dates=["time_utc"])
    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz="CET",
        seed=42,
        test_days= n_days_test , # Here it should be 1*365 for 1 year of test, and so on
        dtype=torch.float64,
    )

    # fix potential problems with dates after change time zone..
    train_dates_series       = pd.DatetimeIndex(sorted(train_dates))
    id_init_exp = train_dates_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_test_period = train_dates_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_exp = train_dates_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t = data_t[id_init_exp:(id_end_exp+1), :,:]
    train_dates = pd.Series(train_dates[id_init_exp:(id_end_exp+1)])

    print(f"days for test data ..{data_t.shape[0] - id_init_test_period}")
    
    maps_dict[code] = {
        "df": df,
        "data_t": data_t,
        "train_t": train_t,
        "train_dates": train_dates,
        "price_t": price_t
    }
maps_dict.keys()

lgbm24_by_zone   = {}
rmse_by_zone  = {}



for z in mapcodes:
    print(f"\n--- {z} ---")
    price_S      = maps_dict[z]["price_t"]
    data_array   = maps_dict[z]["data_t"]
    full_dates      = maps_dict[z]["train_dates"] # <-- here i change a bit the previuous function, prepare_data_tensor, it has all dates
    feature_names= maps_dict[z]["df"].columns[1:]
    full_date_series = pd.DatetimeIndex(sorted(full_dates))

    # evaluation days (all of 2024)
    train_start_idx = full_date_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_eval = full_date_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_eval = full_date_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    eval_start_idx = id_init_eval
    eval_end_idx  = id_end_eval
    N_s            = (eval_end_idx - eval_start_idx) + 1 # additional adjustment
    full_dates = pd.to_datetime(full_dates)

    

    # new features: WD - dummy for week days, price lags for Mon, Tue and Fri, day-ahead load lag
    WD             = [1,6,7]     
    PRICE_S_LAGS   = [1,2,7]
    DA_LAG         = [0]
    S              = 24
    #D             = 730

    # tensors to collect forecasts for THIS zone
    forecasts_zone = torch.full((N_s, S, 1), float("nan"),
                                dtype=torch.float64, device=device)

    # thread pool
    with ThreadPoolExecutor(max(1, os.cpu_count() // 2) + 1 ) as executor:
        futures = [
            executor.submit(
                run_forecast_step_modified,
                n,
                price_S,
                data_array,
                train_start_idx = train_start_idx,
                train_end_idx   = id_init_eval - 1,
                full_dates      = full_dates,
                wd              = WD,                
                price_s_lags    = PRICE_S_LAGS,
                da_lag          = DA_LAG,
                feature_names   = feature_names,   # reg_names
                n_trials_lgbm   = 10,
                days_for_st_model = 14
            )
            for n in range(N_s)
        ]

        for fut in as_completed(futures):
            try:
                n, lgbm24 = fut.result()
                forecasts_zone[n, :, 0] = lgbm24.detach().clone().to(forecasts_zone.dtype).to(forecasts_zone.device)
            except Exception as e:
                import traceback
                print(f"Thread crashed: {e}")
                traceback.print_exc()

                
    #   shape: (N_s, S)
    true_vals = price_S[eval_start_idx : eval_end_idx + 1].to(device)  
    
    # compute RMSE
    diff = forecasts_zone[:, :, 0] - true_vals
    rmse = torch.sqrt((diff**2).mean()).item()
    
    print(f"Zone {z} LBGM-24h RMSE: {rmse:.4f}")

    lgbm24_by_zone[z] = forecasts_zone[:, :, 0].cpu()
    rmse_by_zone[z]  = rmse

[I 2025-06-13 19:50:34,849] A new study created in memory with name: no-name-4dbd72fa-747d-452d-bafb-12973030f914
[I 2025-06-13 19:50:34,855] A new study created in memory with name: no-name-f5e2a22a-3f8e-4b0c-bd7f-e8ffc41282e5
[I 2025-06-13 19:50:34,859] A new study created in memory with name: no-name-406de124-75b6-465e-9c11-8abcead1c319
[I 2025-06-13 19:50:34,899] A new study created in memory with name: no-name-e611242c-01e2-4b5d-af8b-8b7f300a7fc7
[I 2025-06-13 19:50:34,901] A new study created in memory with name: no-name-f6cd9e4b-f7f0-4721-9a23-2f8cfa7cd8b8
[I 2025-06-13 19:50:34,901] A new study created in memory with name: no-name-a944fe89-2f6d-480b-858c-ecdb3ce25477
[I 2025-06-13 19:50:34,919] A new study created in memory with name: no-name-64ab59a3-dff6-436d-816b-e3f120ff3ab4


days for test data ..731

--- NO1 ---
Loop   0: train 2019-01-01 00:00:00 -> 2022-12-31 00:00:00, forecast 2023-01-01 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   1: train 2019-03-03 00:00:00 -> 2023-01-01 00:00:00, forecast 2023-01-02 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   2: train 2019-03-04 00:00:00 -> 2023-01-02 00:00:00, forecast 2023-01-03 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   3: train 2019-03-05 00:00:00 -> 2023-01-03 00:00:00, forecast 2023-01-04 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   4: train 2019-03-06 00:00:00 -> 2023-01-04 00:00:00, forecast 2023-01-05 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   5: train 2019-03-07 00:00:00 -> 2023-01-05 00:00:00, forecast 2023-01-06 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Lo

[I 2025-06-13 19:50:35,730] Trial 0 finished with value: 0.0041884026276050096 and parameters: {'n_estimators': 170, 'learning_rate': 0.2267171607312263, 'num_leaves': 208, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.8347901307909564, 'colsample_bytree': 0.7518650066847192, 'reg_alpha': 9.416125662030684, 'reg_lambda': 3.0000789707017583, 'min_split_gain': 0.010642544956534206}. Best is trial 0 with value: 0.0041884026276050096.
[I 2025-06-13 19:50:35,791] Trial 0 finished with value: 0.005451463170546275 and parameters: {'n_estimators': 191, 'learning_rate': 0.2644163904356763, 'num_leaves': 75, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.9194287334216729, 'colsample_bytree': 0.9750322774697255, 'reg_alpha': 4.3625089613815335, 'reg_lambda': 0.8022076421923785, 'min_split_gain': 0.01542167541511112}. Best is trial 0 with value: 0.005451463170546275.


[Trial 0] New best: 0.00419
[Trial 0] New best: 0.00545


[I 2025-06-13 19:50:36,409] Trial 0 finished with value: 0.01085026440044603 and parameters: {'n_estimators': 102, 'learning_rate': 0.03446360082143626, 'num_leaves': 36, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.5271332603875797, 'colsample_bytree': 0.5596995180927251, 'reg_alpha': 9.163779913858331, 'reg_lambda': 2.9046710469211243, 'min_split_gain': 0.025548136770605495}. Best is trial 0 with value: 0.01085026440044603.
[I 2025-06-13 19:50:36,515] Trial 1 finished with value: 0.004027096412281093 and parameters: {'n_estimators': 203, 'learning_rate': 0.2158775866161607, 'num_leaves': 153, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.7847115425660451, 'colsample_bytree': 0.8038163945949109, 'reg_alpha': 5.527429170026702, 'reg_lambda': 5.418789043215156, 'min_split_gain': 0.0234194654000677}. Best is trial 1 with value: 0.004027096412281093.
[I 2025-06-13 19:50:36,612] Trial 0 finished with value: 0.00740731195870074 and parameters: {'n_estimators': 215, 'lea

...

[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3


[I 2025-06-13 19:52:32,548] Trial 7 finished with value: 0.013413368887839968 and parameters: {'n_estimators': 173, 'learning_rate': 0.021868874899687266, 'num_leaves': 197, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.8390319302216225, 'colsample_bytree': 0.6620593661482164, 'reg_alpha': 5.30241115337457, 'reg_lambda': 4.021758906918738, 'min_split_gain': 0.02649869220735227}. Best is trial 4 with value: 0.010963881440292788.
[I 2025-06-13 19:52:32,553] A new study created in memory with name: no-name-3759994f-7cc8-47a1-b42e-04f97ad13c3a


[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.


[I 2025-06-13 19:52:32,840] Trial 0 finished with value: 0.014814239852215791 and parameters: {'n_estimators': 140, 'learning_rate': 0.10535337759428624, 'num_leaves': 69, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.8533286666525184, 'colsample_bytree': 0.6579277821290797, 'reg_alpha': 3.1378435084943614, 'reg_lambda': 8.81297135725278, 'min_split_gain': 0.04111366719273035}. Best is trial 0 with value: 0.014814239852215791.


[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
             feature  importance
0        Price_lag_1         413
1        Price_lag_2         373
3      Load_DA_lag_0         291
2        Price_lag_7         191
5    WindOn_DA_lag_0         187
11        NGas_lag_2         172
10        Coal_lag_2         167
13         EUA_lag_2         113
12         Oil_lag_2          97
9               WD_7           7
7               WD_1           2
4     Solar_DA_lag_0           0
6   WindOff_DA_lag_0           0
8               WD_6           0
short model ************************
             feature  importance
0        Price_lag_1          77
3      Load_DA_lag_0          18
13         EUA_lag_2           8
1        Price_lag_2           7
2        Price_lag_7           1
7               WD_1           1
12         Oil_lag_2           1
4     Solar_DA_lag_0           0
5    WindOn_DA_lag_0        